In [398]:
import numpy as np

class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, beta=1, linearity=1, alpha=1, deadband=0, 
                 sp_high=100, sp_low=0, gamma=0):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self._itime = itime
        if self._itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.g = gamma

        self.bump = 0
        self.last_output = 0
        self.last_err = 0
        self.last_gerr = 0
        self.last_pv = 0
        self.b = beta
        self.l = linearity
        self.a = alpha
        self.deadband = deadband
        
        self.sp_high = sp_high
        self.sp_low = sp_low
        
        self.Uk = self.Up = self.Ui = self.Ud = 0
        
    @property
    def itime(self):
        return self._itime

    @itime.setter
    def itime(self, v):
        self._itime = v
        if v:
            self.oneoveritime = 1 / v
        else:
            self.oneoveritime = 0

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        uk0 = self.pgain * err_for_pgain
        self.Ui = op - uk0
        self.last_pv = pv
        self.last_err = 0
        self.last_ierr = 0
        self.last_gerr = self.g * sp - pv
        self.last_berr = self.b * sp - pv
        
    def auto_to_auto(self, pv, op):
        self.man_to_auto(pv, pv, op)
        
    set_bump = man_to_auto

    def reset(self):
        self.Ui = 0
        self.last_pv = 0
        self.last_err = 0
        self.last_gerr = 0
    
    def step(self, pv, sp):
        
        err = sp - pv
        berr = self.b * sp - pv
        gerr = self.g * sp - pv
        
        # Based on Labview's `PID Get Advanced Error (DBL).vi`
        if self.l == 1 or self.sp_high == self.sp_low:
            nl_int_factor = 1
        else:
            assert False, "Functionality not tested"
            sp_rng = self.sp_high - self.sp_low
            oml = 1 - self.l
            nl_int_factor = 1 / (1+err*err/ (sp_rng*sp_rng))
            err = err * (self.l + oml*abs(err)/sp_rng)
            berr = berr * (self.l + oml*abs(berr)/sp_rng)
            gerr = gerr * (self.l + oml*abs(gerr)/sp_rng)
        
        errs_avg = (err + self.last_err) / 2
        ierr = errs_avg * nl_int_factor * self.pgain * self.oneoveritime
        
        Up = self.pgain * berr
        Ui = self.Ui = self.Ui + ierr
        
        # Untested
        # This is based off of the code in labview's 
        # PID advanced algorithm, to the best of my
        # ability to decipher the code. 
        # It is probably wrong, since it doesn't match the equation 
        # provided by the manual. 
        
        if 0 < self.a <= 1:
            Ud = (self.Ud - ((self.pgain / self.a) * (gerr - self.last_gerr))) * (self.dtime / (self.dtime + (1 / self.a)))
        else:
            Ud = -(self.last_gerr - gerr) * self.pgain * self.dtime 
        Uk = Up + Ui + Ud
        
        # Coercion & back calculation      
        if Uk > self.auto_max:
            Uk = self.auto_max
            Ui = Uk - Up - Ud
        elif Uk < self.auto_min:
            Uk = self.auto_min
            Ui = Uk - Up - Ud
            
        # XXX debugging
        self.Ui = Ui
        self.Uk = Uk
        self.Ud = Ud
        self.Up = Up
            
        self.last_output = Uk
        self.last_pv = pv
        self.last_err = err
        self.last_ierr = ierr
        self.last_gerr = gerr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.Ui -= 1  # counteract effect of increasing set point on Up
        res_op = p.step(pv, sp)
        assert_equal(p.last_err, sp)
        assert_equal(p.Uk, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [399]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

seconds = 1
minutes = 60 * seconds
hours = 60 * minutes
days = 24 * hours

from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]

In [400]:
import numpy as np
class TempProcess():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.tdelay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv
    
class Delay():
    def __init__(self, delay, initial, pc_at_delay=0.95):
        assert 0 < pc_at_delay <= 1, "don't be a retard"
        if delay:
            decay = 1 - (1-pc_at_delay)**(1/delay)
            self.df = decay
        else:
            self.df = 1
        self.sink = initial / self.df  - initial # fill the sink
        if self.sink < 0:
            self.sink = 0
        
    def delay(self, op):
        self.sink += op
        dNdt = self.df * self.sink
        self.sink -= dNdt
        return dNdt
        
    
class TempProcess2():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = Delay(delay, initial, 0.95)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.heatsink.delay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv

In [401]:
def paste(cells, data, offset=0):
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, 4+offset)).Value = [("T", "PV", "OP", "DPV")]
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [402]:
from officelib.xllib import *
xl = Excel()

try:
    wb
except NameError:
    wb = xl.Workbooks.Add()

try:
    ws = wb.Worksheets(1)
    ws2 = wb.Worksheets(2)
except Exception:
    wb = xl.Workbooks.Add()
    ws = wb.Worksheets.Add()
    ws2 = wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [474]:
def temp_sim(ops):

    # unpack (lazy refactor)
    pv = ops.initial_pv
    sp = ops.setpoint
    op = ops.initial_output
    mode = ops.mode
    k = ops.k
    g = ops.g
    delay = ops.delay
    end = ops.end
    
    pid = PIDController(pgain=ops.p, itime=m2s(ops.i), dtime=m2s(ops.d), auto_max=ops.auto_max,
                        auto_min=ops.auto_min, beta=ops.beta, linearity=ops.linearity, alpha=ops.alpha,
                        deadband=0, sp_high=100, sp_low=0, gamma=ops.gamma)

    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    elif mode == 'o2a':
        pid.off_to_auto(pv, sp)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = ops.process(delay, op, env=ops.env, g=g, k=k)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t/3600, pv, op))
        if t >= end:
            break
    t, pv, op = list(zip(*data))
    dpvdt = np.gradient((np.gradient(pv, 1/3600)), 1/3600) ** 2
    #dpvdt = np.abs(np.gradient(pv, 1/3600))
    data = list(zip(t, pv, op, dpvdt))
    return data

def temp_sim2(ops):
    
    
    pid = PIDController(ops.p,m2s(ops.i),m2s(ops.d),ops.auto_max,ops.auto_min, beta=ops.beta)
    
    pv = ops.initial_pv
    sp = ops.setpoint
    op = ops.initial_output
    mode = ops.mode
    k = ops.k
    g = ops.g
    delay = ops.delay
    end = ops.end
    decay = ops.decay
    decay_time = ops.decay_time
    end2 = ops.end2
    
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = ops.process(delay, op, g=g, k=k, env=ops.env)
    t = 0
    data = [(t/3600, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t/3600, pv, op))
        if t >= end:
            break
    decr = decay / decay_time
    for _ in range(decay_time):
        t += 1
        op = pid.step(pv, sp)
        pv -= decr
        pv = proc.step(pv, op)
        data.append((t/3600, pv, op))
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t/3600, pv, op))
        if t >= end + end2 + decay_time:
            break
    return data
    
def temp_sim3(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37, 
             g=0.0019254, k=-0.001579031, mode='m2a', beta=1, pv_delay=240):
    pid = PIDController(p,m2s(i),m2s(d),amax,amin, do_ifactor=True, beta=beta)
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = TempProcess3(delay, op, g=g, k=k, pv_delay=pv_delay, initial_pv=pv)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end:
            break
    return data
    

In [458]:
from pysrc.snippets import OptionCategory

offset = 0
p = 60
i = 35
data = []
delay=m2s(15)

class TempOps(OptionCategory):
    # PID 
    p = 60
    i = 35
    d = 0
    alpha = -1
    linearity = 1
    beta = 1
    gamma = 0
    auto_max = 100
    auto_min = 0
    
    # plant    
    k = -0.00357  # C / min * C
    g = 0.002053  # C / min * % 
    delay = m2s(5)
    
    # Simulation
    initial_pv = 20
    env = 20
    initial_output = 0
    setpoint = 37
    end = 5 * hours
    mode = 'o2a'
    process = TempProcess2
    
    # For decay simulation only
    decay = 0       # total, C
    decay_time = 0  # total to reach above decay
    end2 = 0        # time to re-stabilize simulation
    

In [459]:
def declare_list(name):
    if name not in globals():
        globals()[name] = []
        
declare_list("data")
declare_list("a2a_data")
declare_list("a2a_data2")

In [406]:
from time import sleep
clear(cells, data)
ws2 = wb.Worksheets("Sheet2")
cells2 = ws2.Cells
cells2.Range("E:I").Clear()
cells2.Range("E2:I2").Value = [("P", "I", "Max T", "Stable Time", "Stable")]
n = 0
current = cells2.Range("E3")

def time_stable(data, at, margin):
    rev = reversed(data)
    hi = at + margin
    lo = at - margin
    time_stable = data[-1][0] - \
            next((t for t, pv, _ in reversed(data) if not (lo <= pv <= hi)), 0)
    return time_stable

def max_t(data):
    max_pv = 0
    max_time = 0
    for t, pv, _ in data:
        if pv > max_pv:
            max_pv = pv
            max_time = t
    return max_pv, max_time

combos = []
for p in range(10, 101, 10):
    for i in range(10, 101, 10):
        combos.append((p, i))
def run_test():
    global data, stable, current
    with HiddenXl(xl):
        for n, (p, i) in enumerate(combos, 1):
            print("\rGenerating data for P:%.2f I:%.2f (Test %d of %d)     " % (p, i, n, len(combos)), end="")
            data = temp_sim2(p=p, i=i, d=0, delay=delay, 
                    amax=100, amin=0, end=m2s(60), op=15.17, 
                    pv=37, sp=37, mode='a2a', beta=0, Klass2=TempProcess2, decay=.002, decay_time=900, end2=m2s(150))
            cells.Range("E1:F1").Value = [(p, i)]
            paste(cells, data, 0)
            current.Value = "%.2f" % p
            current.Offset(1, 2).Value = "%.2f" % i
            current.Offset(1, 3).Value = xl.WorksheetFunction.Max(cells.Range("B:B"))
            stable = time_stable(data, 37, 0.05)
            current.Offset(1, 4).Value = stable
            current.Offset(1, 5).Value = stable > 3600
            current = current.Offset(2, 1)

In [407]:
def summarize(cells, data, ops, margin, n):
    header = [("PGain", "Itime", "DTime", "Mode", 
               "Start", "Setpoint", 
               "Settle Time", "Max", "Sum Derivative")]
    if not cells(1,1).Value:
        cells.Range(cells(1,1), cells(1, len(header[0]))).Value = header
    rev = reversed(data)
    hi = ops.setpoint + margin
    lo = ops.setpoint - margin
    settle_time = next((t for t, pv, *_ in rev if not (lo <= pv <= hi)), 0)
    max_temp = max(pv for _, pv, *_ in data)
    sd = sum(dpvs for t,_,_,dpvs in data if 1 < t < 2) / 3600
    target_range = cells.Range(cells(1+n, 1), cells(1+n, len(header[0])))
    target_range.Value = [(ops.p, ops.i, ops.d, ops.mode, 
                           ops.initial_pv, ops.setpoint, 
                           settle_time, max_temp, sd)]

In [460]:
ops = TempOps()

# PID
ops.p = 70
ops.i = 15
ops.d = 0
ops.alpha = -1
ops.linearity = 1
ops.beta = 0
ops.gamma = 0
ops.auto_max = 50
ops.auto_min = 0

# plant    
ops.k = -0.00357 * 1    # C / min * C
ops.g = 0.00205296  * 2.1 # C / min * % 
ops.delay = m2s(12)

# Simulation
ops.initial_pv = 24
ops.env = 21.972
ops.initial_output = 0
ops.setpoint = 37
ops.end = 5 * hours
ops.mode = 'o2a'
ops.process = TempProcess2

clear_data()

with screen_lock(xl):
    data = temp_sim(ops)
    paste(cells, data, 0)
    summarize(ws2.Cells, data, ops, 0.05, n+1)

    ops.mode = 'a2a'
    ops.initial_pv = 35
    ops.initial_output = 11.6
    a2a_data = temp_sim(ops)
    paste(cells, a2a_data, 6)
    summarize(ws2.Cells, a2a_data, ops, 0.05, n+2)

    ops.initial_pv = 35.8
    a2a_data2 = temp_sim(ops)
    paste(cells, a2a_data2, 12)
    summarize(ws2.Cells, a2a_data2, ops, 0.05, n+3)
    
# reset initial pv so below cells don't fuck up
ops.initial_pv = 24

In [411]:
2*0.00205296, 2.1*0.00205296, 1.05 * 0.00410592

(0.00410592, 0.004311216, 0.004311216)

In [475]:
def clear_data(ws=ws):
    with screen_lock(xl):
        for c in ws.UsedRange.Columns:
            if c.Column not in range(5,14):
                c.EntireColumn.Clear()

def compare_values(values, param, process=temp_sim):
    global chart, chart2, data
        
    with screen_lock(xl):
        
        c = 1
        ws3 = wb.Worksheets("Sheet3")
        ws3.UsedRange.Clear()

        chart = ws3.ChartObjects(1).Chart
        chart2 = ws3.ChartObjects(2).Chart
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        cells = ws3.Cells


        for v in values:
            setattr(ops, param, v)
            data = process(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "%s=%.2g"%(param, v))
            CreateDataSeries(chart2, xr, yr2, "%s=%.2g"%(param,v))
            
            c += 4
            
def compare_values2(values, param, base, label=None):
    global chart, chart2, data
    if label is None:
        label = param
    with screen_lock(xl):
        clear_data()
        c = 1
        chart = wb.Charts(2)
        chart2 = wb.Charts(3)
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        xr = cells.Range("H8:H1106")
        yr = cells.Range("F8:F1106")
        yr2 = cells.Range("G8:G1106")
        CreateDataSeries(chart, xr, yr, "Actual")
        CreateDataSeries(chart2, xr, yr2, "Actual")

        for v in values:
            setattr(ops, param, v * base)
            data = temp_sim(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "%s=%.3g"%(label, v))
            CreateDataSeries(chart2, xr, yr2, "%s=%.3g"%(label,v))
            c += 5
            if c == 6:
                c = 10
                

def compare_values3(args, labels):
    global chart, chart2, data
    with screen_lock(xl):
        clear_data()
        c = 1
        chart = wb.Charts(2)
        chart2 = wb.Charts(3)
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        xr = cells.Range("H8:H1106")
        yr = cells.Range("F8:F1106")
        yr2 = cells.Range("G8:G1106")
        CreateDataSeries(chart, xr, yr, "Actual")
        CreateDataSeries(chart2, xr, yr2, "Actual")
        
        values, params, bases, operations = list(zip(*args))
        for *vs, l in zip(*values, labels):
            for v, p, base, op in zip(vs, params, bases, operations):
                if op == "*":
                    val = base * v
                elif op == "=":
                    val = v
                elif op == "+":
                    val = base + v
                elif op == "-":
                    val = base - v
                elif op == "--":
                    val = v - base
                elif op == "/":
                    val = base / v
                elif op == "\\":
                    val = v / base
                else:
                    raise ValueError(op)
                setattr(ops, p, base * v)
            data = temp_sim(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, l)
            CreateDataSeries(chart2, xr, yr2, l)
            
            c += 4
            if c == 5:
                c = 10
    

In [257]:
k = -0.00357  # C / min * C
g = 0.00205296  * 1.05 # C / min * % 
ops.auto_max = 50
kms = [i/100 for i in range(110, 150, 20)]
gms = [m2s(i) for i in range(5, 16, 5)]
op = "*"

combos = [(_k, _g) for _k in kms for _g in gms]
kms, gms = list (zip(*combos))

gms = [g*2 for g in gms]

labels = ["km = %.2f d = %.2f" % (k, g/60) for k, g in zip(kms, gms)]

args = [(kms, 'k', k, op), (gms, 'd', 0, "=")]
compare_values3(args, labels)

In [201]:
# impact of process decay constant multiplier on heat ramp
values = [(1+v/100) for v in range(-40, 1, 20)]
compare_values2(values, "k", -0.00357, 'Km')

In [241]:
# impact of process gain constant multiplier on heat ramp
values = [(1+cm/100) for cm in range(80, 111, 20)]
compare_values2(values, "g", 0.002053, "gm")

In [94]:
ops.delay=m2s(7)
values = [g/100 for g in (0.178437, 0.205296, 0.151579)]
compare_values(values, 'g')

In [242]:
# impact of process gain constant multiplier on heat ramp
values = [(1-cm/100) for cm in range(-10, 11, 5)]
values = (2.1,)
compare_values2(values, "g", 0.002053, "gm")

In [466]:
ops.initial_pv = 37
ops.mode = 'a2a'
ops.initial_output = 12.44
ops.end = 1000
ops.p = 70
ops.i = 20
clear(cells, data, 0)
ops.decay = 0.2
ops.decay_time = 700
ops.end2 = m2s(60)
data = temp_sim2(ops)
paste(cells, data, 0)

In [486]:
decays = [i/10 for i in range(1,11, 2)]
ops.p = 70
ops.i = 15
ops.auto_max=50
ops.decay_time = 480
ops.end2 = m2s(60)
compare_values(decays, 'decay', temp_sim2)

In [394]:
def compare_values4(combos):
    global chart, chart2, data
    ws3 = wb.Worksheets("Sheet3")
    cells = ws3.Cells
    with screen_lock(xl):
        ws3.UsedRange.Clear()
        c = 1
        chart = ws3.ChartObjects(1).Chart
        chart2 = ws3.ChartObjects(2).Chart
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        ntests = 2 * len(combos)
        n = 1
        for cb in combos:
            p, i = cb
            ops.p = p
            ops.i = i
            ops.mode = 'o2a'
            ops.initial_pv = 24
            print("\rRunning test %d of %d" % (n, ntests), end="")
            data = temp_sim(ops)
            paste(ws3.Cells, data[::15], c-1)
            summarize(cells2, data, ops, 0.05, n)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "P:%d I:%d"%(p,i))
            
            n += 1
            c += 6

            ops.mode = 'a2a'
            ops.initial_pv = 35
            data = temp_sim(ops)
            paste(ws3.Cells, data[::15], c-1)
            summarize(cells2, data, ops, 0.05, n)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            CreateDataSeries(chart2, xr, yr, "P:%d I:%d"%(p,i))
            
            n += 1
            c += 6
        print("\rDone                                           ")

In [395]:
cells2 = ws2.Cells
ws2.UsedRange.Clear()
n = 1
combos = [(p, i) for p in range(60, 71, 10) for i in range(10, 26, 5)]

compare_values4(combos)

Done                                           


In [507]:
compare_values4(((70, 20), (70, 15)))

Done                                           


# Matplotlib Outputs Below

In [588]:
ops = TempOps()

# PID
ops.p = 70
ops.i = 20
ops.d = 0
ops.alpha = -1
ops.linearity = 1
ops.beta = 0
ops.gamma = 0
ops.auto_max = 50
ops.auto_min = 0

# plant    
ops.k = -0.00357 * 1    # C / min * C
ops.g = 0.00205296  * 2.1 # C / min * % 
ops.delay = m2s(12)

# Simulation
ops.initial_pv = 24
ops.env = 21.972
ops.initial_output = 0
ops.setpoint = 37
ops.end = 3 * hours
ops.mode = 'o2a'
ops.process = TempProcess2

data = temp_sim(ops)
ops.i = 15
data2 = temp_sim(ops)

def arrays(data):
    return [np.array(a) for a in list(zip(*data))]

%matplotlib
import matplotlib.pyplot as plt
import numpy as np
def plot(datas, labels, title):
    global fig, ax
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for d, l in zip(datas, labels):
        t, pv, op, _ = arrays(d)
        ax.plot(t, pv, ls='-', label=l)
        
    ax.legend(loc="lower center")
    ax.grid()
    b = ax.get_position()
    ax.set_position([b.x0, b.y0+0.1, b.width, b.height*0.9])

    ax.legend(bbox_to_anchor=(0., -0.23, 1., .102), loc="center",
               ncol=max(len(labels), 5), mode="expand", borderaxespad=0.)

    ax.xaxis.set_label_text("Time(hr)")
    ax.yaxis.set_label_text("Temp(C)")
    fig.suptitle(title, fontsize=20)

Using matplotlib backend: TkAgg


In [597]:
def test_pi1(p, i, pv, sp, op):
    ops.p = p
    ops.i = i
    ops.initial_pv = pv
    ops.setpoint = sp
    ops.initial_output = op
    return temp_sim(ops)

def test_pi(p, i):
    data = test_pi1(p, i, 37, 35, 12.4669)
    data2 = test_pi1(p, i, 35, 37, data[-1][2])
    return data, data2
ops.env = 30
d1, d2 = test_pi(70, 20)
d3, d4 = test_pi(70, 15)
l1 = "P:70 I:20"
l2 = "P:70 I:15"
labels = [l1, l1, l2, l2]
data = [d1, d2, d3, d4]
plot(data, labels, "Simulated Set Point Change")

ax.set_ylim((34.5, 37.5))
ax.set_xlim((0, 2.5))

(0, 2.5)

In [593]:
ops.mode = "a2a"
stuff = [(test_pi1(70, 15, 37-i/10, 37, 12.444), "%.2f\u00B0C"%(37-i/10)) for i in range(4,21,4)]
data, labels = list(zip(*reversed(stuff)))
plot(data, labels, "Setpoint Change Magnitude Comparison")
ax.set_ylim((34.5, 37.2))
ax.set_xlim((0, 2.5))

(0, 2.5)